# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_weather = pd.read_csv("output_data/cities.csv")
city_weather.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,qaanaaq,77.4840,-69.3632,16.09,65,60,7.76,GL,1630971841
1,ushuaia,-54.8000,-68.3000,44.26,76,40,12.66,AR,1630971754
2,riyadh,24.6877,46.7219,86.14,13,0,3.44,SA,1630971841
3,khatanga,71.9667,102.5000,53.29,67,93,8.57,RU,1630971843
4,bluff,-46.6000,168.3333,50.81,90,100,6.33,NZ,1630971843


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
figure_layout = {'width': '600px', 'height': '440px'}
fig = gmaps.figure(layout=figure_layout)
heatmap_layer=gmaps.heatmap_layer(city_weather[['Lat', 'Lng']], weights=city_weather['Humidity'], 
                                  max_intensity=100, point_radius=9)
heatmap_layer.opacity = .75
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='440px', width='600px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
temps_best = city_weather.loc[city_weather['Max Temp'] > 70 ]
humidity_best = temps_best.loc[temps_best['Humidity'] < 65]
wind_best = humidity_best.loc[humidity_best['Wind Speed'] < 5]
hotel_df = wind_best.loc[wind_best['Cloudiness'] < 10 ]
hotel_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,riyadh,24.6877,46.7219,86.14,13,0,3.44,SA,1630971841
41,wana,32.2989,69.5725,77.56,57,0,2.73,PK,1630971858
90,arinos,-15.9169,-46.1056,80.46,24,3,1.81,BR,1630971879
108,doha,25.2867,51.5333,93.18,36,0,3.44,QA,1630971846
139,chikwawa,-16.0335,34.8009,73.31,53,1,0.22,MW,1630971898
146,kumluca,36.3703,30.2869,78.12,53,0,4.03,TR,1630971902
190,korla,41.7597,86.1469,70.74,29,0,1.57,CN,1630971922
215,saint george,37.1041,-113.5841,104.47,12,1,1.01,US,1630971933
301,vidalia,32.2177,-82.4135,86.11,61,1,0.00,US,1630971968
336,albion,42.2431,-84.7530,74.64,60,1,0.00,US,1630971983


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df.insert(9, "Hotel Name", "")
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
2,riyadh,24.6877,46.7219,86.14,13,0,3.44,SA,1630971841,
41,wana,32.2989,69.5725,77.56,57,0,2.73,PK,1630971858,
90,arinos,-15.9169,-46.1056,80.46,24,3,1.81,BR,1630971879,
108,doha,25.2867,51.5333,93.18,36,0,3.44,QA,1630971846,
139,chikwawa,-16.0335,34.8009,73.31,53,1,0.22,MW,1630971898,
146,kumluca,36.3703,30.2869,78.12,53,0,4.03,TR,1630971902,
190,korla,41.7597,86.1469,70.74,29,0,1.57,CN,1630971922,
215,saint george,37.1041,-113.5841,104.47,12,1,1.01,US,1630971933,
301,vidalia,32.2177,-82.4135,86.11,61,1,0.00,US,1630971968,
336,albion,42.2431,-84.7530,74.64,60,1,0.00,US,1630971983,


In [9]:
target_radius = 5000
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": target_radius,
    "type": "lodging",
    "key": g_key,
}

# Iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    # Get lat and lng from dataframe
    lat_lon = str(row['Lat']) + "," + str(row['Lng'])

    # Add keyword to params dict
    params['location'] = lat_lon

    # Assemble url and make API request
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    # Get results
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    except:
        print('Error')

/Users/bodensteiner/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [10]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
2,riyadh,24.6877,46.7219,86.14,13,0,3.44,SA,1630971841,"InterContinental Riyadh, an IHG Hotel"
41,wana,32.2989,69.5725,77.56,57,0,2.73,PK,1630971858,Rest House at Wana Camp
90,arinos,-15.9169,-46.1056,80.46,24,3,1.81,BR,1630971879,casa do bruno
108,doha,25.2867,51.5333,93.18,36,0,3.44,QA,1630971846,Movenpick Hotel Doha
139,chikwawa,-16.0335,34.8009,73.31,53,1,0.22,MW,1630971898,Thawale Lodge
146,kumluca,36.3703,30.2869,78.12,53,0,4.03,TR,1630971902,TurkuazKöy
190,korla,41.7597,86.1469,70.74,29,0,1.57,CN,1630971922,Gulisitan Hotel
215,saint george,37.1041,-113.5841,104.47,12,1,1.01,US,1630971933,Best Western Plus Abbey Inn
301,vidalia,32.2177,-82.4135,86.11,61,1,0.00,US,1630971968,Quality Inn & Suites
336,albion,42.2431,-84.7530,74.64,60,1,0.00,US,1630971983,Days Inn by Wyndham Albion


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
figure_layout = {'width': '600px', 'height': '440px'}
fig = gmaps.figure(layout=figure_layout)

heatmap_layer = gmaps.heatmap_layer(city_weather[['Lat', 'Lng']], weights=city_weather['Humidity'], 
                                  max_intensity=100, point_radius=9)

# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

heatmap_layer.opacity = .75

fig.add_layer(heatmap_layer)
fig.add_layer(marker_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='440px', width='600px'))